In [1]:
from django.template import Context

from django.template.base import Token
from django.template.base import Parser
from django.template.base import Template
from django.template.base import TokenType

from django.core.management import call_command

from wagtail_srcset.templatetags.wagtail_srcset_tags import srcset_image

In [2]:
from django.core.files.uploadedfile import SimpleUploadedFile

from wagtail.images.models import Image as WagtailImage

# setup db

In [3]:
call_command("migrate")

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions, sites, taggit, wagtailadmin, wagtailcore, wagtaildocs, wagtailembeds, wagtailforms, wagtailimages, wagtailredirects, wagtailsearch, wagtailusers
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applyin

# create image

In [4]:
import io

from PIL import Image



def create_small_rgb():
    # this is a small test jpeg
    img = Image.new('RGB', (200, 200), (255, 0, 0, 0))
    return img


def small_jpeg_io():
    rgb = create_small_rgb()
    im_io = io.BytesIO()
    rgb.save(im_io, format="JPEG", quality=60, optimize=True, progressive=True)
    im_io.seek(0)
    im_io.name = "testimage.jpg"
    return im_io


def small_uploaded_file(small_jpeg_io):
    simple_png = SimpleUploadedFile(
        name="test.png", content=small_jpeg_io.read(), content_type="image/png"
    )
    small_jpeg_io.seek(0)
    return simple_png

In [5]:
simple_png = small_uploaded_file(small_jpeg_io())

In [6]:
from django.conf import settings

print(settings.DATABASES)

{'default': {'ENGINE': 'django.db.backends.sqlite3', 'NAME': ':memory:', 'ATOMIC_REQUESTS': False, 'AUTOCOMMIT': True, 'CONN_MAX_AGE': 0, 'OPTIONS': {}, 'TIME_ZONE': None, 'USER': '', 'PASSWORD': '', 'HOST': '', 'PORT': '', 'TEST': {'CHARSET': None, 'COLLATION': None, 'NAME': None, 'MIRROR': None}}}


In [7]:
image = WagtailImage(file=simple_png)
image.save()

# render template

In [8]:
template_text = """
{% load wagtailimages_tags %}
{% load wagtail_srcset_tags %}

{% image img width-300 %}
{% srcset_image img width-300 %}
"""

In [9]:
t = Template(template_text)
print(t.render(Context({"img": image})))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'attrs', 'child_nodelists', 'filter', 'filter_spec', 'get_nodes_by_type', 'image_expr', 'must_be_first', 'output_var_name', 'render', 'render_annotated', 'token']




<img alt="" height="200" src="/media/images/test_G5BCc9J.width-300.jpg" width="200">
<img alt="" height="200" src="/media/images/test_G5BCc9J.width-300.jpg" srcset="/media/images/test_G5BCc9J.width-2200.jpegquality-60.jpg 200w, /media/images/test_G5BCc9J.width-1100.jpegquality-60.jpg 200w, /media/images/test_G5BCc9J.width-768.jpegquality-60.jpg 200w, /media/images/test_G5BCc9J.width-500.jpegquality-60.jpg 200w, /media/images/test_G5BCc9J.width-300.jpegquality-60.jpg 200w"

In [17]:
template_text = """
{% load wagtailimages_tags %}

{% image img width-300 %}
"""

In [18]:
t = Template(template_text)
t.render(Context({"img": image}))

'\n\n\n<img alt="" height="200" src="/media/images/test.width-300.jpg" width="200">\n'

In [13]:
image_tag = "{% image block.value width-300 %}"
image_tag = "block.value width-300}"

token = Token(TokenType.BLOCK, image_tag)
parser = Parser(token.split_contents())

In [19]:
t = Template(template_text)
t.render(Context({}))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'attrs', 'child_nodelists', 'filter', 'filter_spec', 'get_nodes_by_type', 'image_expr', 'must_be_first', 'output_var_name', 'render', 'render_annotated', 'token']


'\n\n\n\n\n\n'

# Get image size in tag

In [23]:
from django import template
from django.conf import settings

from wagtail.images.templatetags.wagtailimages_tags import image

In [25]:
register = template.Library()


@register.tag(name="srcset_image2")
def srcset_image(parser, token):
    image_node = image(parser, token)
    print(image_node)
    print(dir(image_node))
    image_node.attrs["srcset"] = SrcSet(image_node)
    return image_node


class SrcSet:
    def __init__(self, image_node):
        self.image_node = image_node
        srcset = image_node.attrs.get("srcset", None)
        print("image node attrs: ", image_node.attrs)
        print("image node width: ", image_node.attrs.get("width"))
        print("image node filter: ", image_node.filter.operations)
        if srcset is None:
            self.renditions = self.default_renditions
        else:
            self.renditions = self.renditions_from_srcset(srcset.token)

    @property
    def default_renditions(self):
        if hasattr(settings, "DEFAULT_SRCSET_RENDITIONS"):
            return settings.DEFAULT_SRCSET_RENDITIONS
        else:
            return [
                "width-2200|jpegquality-60",
                "width-1100|jpegquality-60",
                "width-768|jpegquality-60",
                "width-500|jpegquality-60",
                "width-300|jpegquality-60",
            ]

    def renditions_from_srcset(self, srcset):
        srcset = srcset.strip('"').strip("'")
        return srcset.split(" ")

    def resolve(self, context):
        image = self.image_node.image_expr.resolve(context)
        out_renditions = []
        for rendition in self.renditions:
            rendered_image = image.get_rendition(rendition)
            out_renditions.append(f"{rendered_image.url} {rendered_image.width}w")
        srcset_string = ", ".join(out_renditions)
        return srcset_string

In [33]:
template_text = """
{% load wagtailimages_tags %}
{% load wagtail_srcset_tags %}


{% image img width-300 %}
{% srcset_image img width-300 %}
"""

In [34]:
t = Template(template_text)
t.render(Context({"img": image}))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'attrs', 'child_nodelists', 'filter', 'filter_spec', 'get_nodes_by_type', 'image_expr', 'must_be_first', 'output_var_name', 'render', 'render_annotated', 'token']


'\n\n\n\n\n\n\n'